In [2]:
from pyspark.sql import SparkSession
import getpass
username=getpass.getuser()
spark=SparkSession.\
    builder.\
    config('spark.ui.port','0').\
    config("spark.sql.warehouse.dir",f"/user/{username}/warehouse").\
    enableHiveSupport().\
    master('yarn').\
    getOrCreate()

In [2]:
spark

In [3]:
rdd1 = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/*")
rdd1.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [ ]:
rdd2 = rdd1.map(lambda x : (x.split(",")[3],1))
rdd2.take(5)

In [5]:
rdd3 = rdd2.reduceByKey(lambda x,y : x+y)


In [6]:
rdd3.collect()

[('CLOSED', 7556),
 ('CANCELED', 1428),
 ('PENDING_PAYMENT', 15030),
 ('COMPLETE', 22899),
 ('PROCESSING', 8275),
 ('PAYMENT_REVIEW', 729),
 ('PENDING', 7610),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558)]

In [7]:
sorted_rdd = rdd3.sortBy(lambda x: x[1])

In [8]:
sorted_rdd.collect()

[('PAYMENT_REVIEW', 729),
 ('CANCELED', 1428),
 ('SUSPECTED_FRAUD', 1558),
 ('ON_HOLD', 3798),
 ('CLOSED', 7556),
 ('PENDING', 7610),
 ('PROCESSING', 8275),
 ('PENDING_PAYMENT', 15030),
 ('COMPLETE', 22899)]

In [12]:
#customer
cust_rdd1 = rdd1.map(lambda x : (x.split(",")[2],1))

In [13]:
cust_rdd1.take(3)

[('11599', 1), ('256', 1), ('12111', 1)]

In [14]:
cust_rdd2 = cust_rdd1.reduceByKey(lambda x,y :x+y)

In [15]:
cust_rdd2.take(3)

[('256', 10), ('12111', 6), ('11318', 6)]

In [16]:
cust_sort_rdd = cust_rdd2.sortBy(lambda x : x[1], False)

In [18]:
cust_sort_rdd.take(10)

[('5897', 16),
 ('6316', 16),
 ('12431', 16),
 ('569', 16),
 ('4320', 15),
 ('221', 15),
 ('5624', 15),
 ('5283', 15),
 ('12284', 15),
 ('5654', 15)]

In [21]:
#customer atleat one
distinct_cust = rdd1.map(lambda x : x.split(",")[2]).distinct()

In [22]:
distinct_cust.count()

12405

In [4]:
#closed orders customer
filter_order = rdd1.filter(lambda x : (x.split(",")[3] == 'CLOSED'))

In [5]:
filter_order.take(4)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '12,2013-07-25 00:00:00.0,1837,CLOSED',
 '18,2013-07-25 00:00:00.0,1205,CLOSED']

In [8]:
closed_order = filter_order.map(lambda x : (x.split(",")[2], 1))

In [9]:
closed_order.take(3)

[('11599', 1), ('8827', 1), ('1837', 1)]

In [10]:
agg_rdd = closed_order.reduceByKey(lambda x,y :x+y)
agg_rdd.take(3)

[('5863', 1), ('12271', 2), ('7073', 1)]

In [12]:
sorted_orders = agg_rdd.sortBy(lambda x : x[1], False)

In [14]:
sorted_orders.take(10)

[('1833', 6),
 ('5493', 5),
 ('1363', 5),
 ('1687', 5),
 ('2768', 4),
 ('5319', 4),
 ('437', 4),
 ('2774', 4),
 ('10111', 4),
 ('5582', 4)]